# Optimised consumption

In [8]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

# plot the results
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import numpy as np

from ortools.linear_solver import pywraplp
import time

## Configurations

In [9]:

########### General
date_range = ('2019-10-01', '2019-11-01')
resolution = 'h' #30min
time_of_use = False
include_ev = False
scenario_name = 'no_ev_40_km_scenario'

# EV usage
ev_daily_avg_usage_km = 40 # km
ev_daily_usage_std = 5 # km

ev_efficiency = 0.18 # kWh/km
ev_charging_start = 20 # 8pm
ev_charging_end = 6 # 6am

# Tariffs
flat_tariff =  24.5# p/kWh
feed_in_tariff = 5 # p/kWh
peak_tariff = 27.6 # p/kWh
off_peak_tariff = 17.1 # p/kWh
peak_initial_hour = 7
peak_final_hour = 24


# Battery configurations
battery_soc_0 = 0
charge_efficiency = 0.95
discharge_efficiency = 0.95


# ev battery configurations
ev_max_power_kw = 2.3 # kW
ev_max_energy_kwh = 40 # kWh
ev_charge_efficiency = 0.95
ev_soc_0 = ev_max_energy_kwh*1000


# Model configurations
M = 1e9
eps =1e-7

## Data import

In [10]:
res_dict ={'h':60,'30min':30,'15min':15,'10min':10,'5min':5,'1min':1}
scale = res_dict[resolution]

df_sites = pd.read_csv('data/t_sites.csv')
df_sites = df_sites[['site','besskwh','besskw','pvkw','majorload']]
df_metadata = pd.read_excel('data/LEM_Resi_MetaData_Tables.xlsx', sheet_name='Household Questionaire')
df_metadata_ev = df_metadata[['newsite']+list(df_metadata.columns[52:60])].dropna(subset=df_metadata.columns[52:60], how='all')

sites_with_ev = df_sites[(df_sites['site'].isin(df_metadata_ev['newsite']) | df_sites['majorload'].str.contains('EV'))]
useful_sites = df_sites[~df_sites['site'].isin(sites_with_ev['site'])]

In [11]:
def get_optimal_solution(site, df_sites, date_range, resolution, time_of_use, ev_daily_avg_usage_km, ev_daily_usage_std, ev_efficiency, ev_charging_start, ev_charging_end, flat_tariff, feed_in_tariff, peak_tariff, off_peak_tariff, peak_initial_hour, peak_final_hour, battery_soc_0, charge_efficiency, discharge_efficiency, ev_max_power_kw, ev_max_energy_kwh, ev_charge_efficiency, ev_soc_0, M, eps, scale):
    ################## Handle data ####################
    besskwh = df_sites[df_sites['site'] == site]['besskwh'].values[0]
    besskw = df_sites[df_sites['site'] == site]['besskw'].values[0]

    df = pd.read_parquet(f'data/t_msb1m/site={site}/', engine='pyarrow')
    df= df[(df['ts']>=date_range[0]) & (df['ts']<date_range[1])]
    df['date_hour'] = df['ts'].dt.floor(resolution)
    df_hourly = df.groupby(['date_hour']).agg({'production_wh':'mean','consumption_wh':'mean'}).reset_index()
    df_hourly['production_wh'] = df_hourly['production_wh']*scale
    df_hourly['consumption_wh'] = df_hourly['consumption_wh']*scale

    # resample to fill missing hours
    df_hourly = df_hourly.set_index('date_hour').resample(resolution).mean().reset_index()

    # fillnas
    df_hourly['production_wh'] = df_hourly['production_wh'].fillna(0)
    df_hourly['consumption_wh'] = df_hourly['consumption_wh'].fillna(0)

    ################## EV usage ####################
    morning_ev_usage_kwh = np.random.normal(ev_daily_avg_usage_km, ev_daily_usage_std, len(df_hourly))*ev_efficiency/2
    evening_ev_usage_kwh = np.random.normal(ev_daily_avg_usage_km, ev_daily_usage_std, len(df_hourly))*ev_efficiency/2
    ev_daily_usage_kwh = morning_ev_usage_kwh + evening_ev_usage_kwh

    ################## Tariff ####################
    tariffs = None
    if time_of_use:
        tariffs = [peak_tariff if df_hourly['date_hour'][i].hour >= peak_initial_hour and df_hourly['date_hour'][i].hour < peak_final_hour else off_peak_tariff for i in range(len(df_hourly))]
    else:
        tariffs = [flat_tariff for i in range(len(df_hourly))]


    ################## Model ####################

    # We first initialise the solver.
    solver = pywraplp.Solver.CreateSolver('SCIP')

    # battery charge variables
    battery_charge = [solver.NumVar(0, besskw*1000, f'battery_charge_{i}') for i in range(len(df_hourly))]
    # battery discharge variables
    battery_discharge = [solver.NumVar(0, besskw*1000, f'battery_discharge_{i}') for i in range(len(df_hourly))]
    # battery state of charge variables
    battery_soc = [solver.NumVar(0, besskwh*1000, f'battery_soc_{i}') for i in range(len(df_hourly))]
    # grid import variables
    grid_import = [solver.NumVar(0, solver.infinity(), f'grid_import_{i}') for i in range(len(df_hourly))]
    # grid export variables
    grid_export = [solver.NumVar(0, solver.infinity(), f'grid_export_{i}') for i in range(len(df_hourly))]

    if include_ev:
        #ev charging
        ev_charge= [solver.NumVar(0, ev_max_power_kw*1000, f'ev_charging_{i}') for i in range(len(df_hourly))]
        #ev soc
        ev_soc = [solver.NumVar(0, ev_max_energy_kwh*1000, f'ev_soc_{i}') for i in range(len(df_hourly))]


    # prevent charge and discharge at the same time
    is_charging = [solver.IntVar(0, 1, f'is_charging_{i}') for i in range(len(df_hourly))]
    # prevent import and export at the same time
    is_importing = [solver.IntVar(0, 1, f'is_importing_{i}') for i in range(len(df_hourly))]

    # penalties to avoid oscilations
    charge_penalty = [solver.NumVar(0, solver.infinity(), f'charge_penalty_{i}') for i in range(len(df_hourly))]
    discharge_penalty = [solver.NumVar(0, solver.infinity(), f'discharge_penalty_{i}') for i in range(len(df_hourly))]
    
    if include_ev:
        ev_charge_penalty = [solver.NumVar(0, solver.infinity(), f'ev_charge_penalty_{i}') for i in range(len(df_hourly))]

    # constraints
    for i in range(len(df_hourly)):

        if include_ev:
            hour = df_hourly['date_hour'][i].hour
            if hour >= ev_charging_start or hour < ev_charging_end:
                if i==0:
                    solver.Add(ev_soc[i] == ev_soc_0+ev_charge[i]*ev_charge_efficiency*(scale/60))
                else:
                    solver.Add(ev_soc[i] == ev_soc[i-1]+ev_charge[i]*ev_charge_efficiency*(scale/60))
            else:
                solver.Add(ev_charge[i] == 0)
                if hour == 12:
                    solver.Add(ev_soc[i] == ev_soc[i-1] - ev_daily_usage_kwh[i]*1000)
                else:
                    if i==0:
                        solver.Add(ev_soc[i] == ev_soc_0)
                    else:
                        solver.Add(ev_soc[i] == ev_soc[i-1])

        if i == 0:
            solver.Add(battery_soc[i] == battery_soc_0 
                                        + battery_charge[i]*charge_efficiency*(scale/60) 
                                        - battery_discharge[i]*(1/discharge_efficiency)*(scale/60))
        else:
            solver.Add(battery_soc[i] == battery_soc[i-1] 
                                        + battery_charge[i]*charge_efficiency*(scale/60) 
                                        - battery_discharge[i]*(1/discharge_efficiency)*(scale/60))
        
        if include_ev:
            solver.Add(grid_export[i] - grid_import[i] + battery_charge[i]*(scale/60)  - battery_discharge[i]*(scale/60) + ev_charge[i]*(scale/60)  
                                            ==  df_hourly['production_wh'][i] - df_hourly['consumption_wh'][i])
        else:
            solver.Add(grid_export[i] - grid_import[i] + battery_charge[i]*(scale/60)  - battery_discharge[i]*(scale/60) 
                       ==  df_hourly['production_wh'][i] - df_hourly['consumption_wh'][i])
        solver.Add(battery_charge[i] <= M * is_charging[i])
        solver.Add(battery_discharge[i] <= M * (1 - is_charging[i]))
        solver.Add(grid_import[i] <= M * is_importing[i])
        solver.Add(grid_export[i] <= M * (1 - is_importing[i]))

        # penalise the variations of is importing and is charging
        if i > 0:
            solver.Add(charge_penalty[i] >= battery_charge[i] - battery_charge[i-1])
            solver.Add(discharge_penalty[i] >= battery_discharge[i] - battery_discharge[i-1])
            if include_ev:
                solver.Add(ev_charge_penalty[i] >= ev_charge[i] - ev_charge[i-1])

    # objective function
    consumption_cost = sum([tariffs[i]*grid_import[i] for i in range(len(df_hourly))])

    if include_ev:
        solver.Minimize(consumption_cost 
                    - sum(grid_export)*feed_in_tariff
                    + eps*sum(charge_penalty)
                    + eps*sum(discharge_penalty)
                    + eps*sum(ev_charge_penalty)
                    )
    else:
        solver.Minimize(consumption_cost 
                    - sum(grid_export)*feed_in_tariff
                    + eps*sum(charge_penalty)
                    + eps*sum(discharge_penalty)
                    )
    
    solver.set_time_limit(60000)  
            
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print(f'Optimal solution found for site {site}')
        # find the optimal values
        battery_charge_values = [battery_charge[i].solution_value() for i in range(len(df_hourly))]
        battery_discharge_values = [battery_discharge[i].solution_value() for i in range(len(df_hourly))]
        battery_soc_values = [battery_soc[i].solution_value() for i in range(len(df_hourly))]
        grid_import_values = [grid_import[i].solution_value() for i in range(len(df_hourly))]
        grid_export_values = [grid_export[i].solution_value() for i in range(len(df_hourly))]
        is_charging_values = [is_charging[i].solution_value() for i in range(len(df_hourly))]
        is_importing_values = [is_importing[i].solution_value() for i in range(len(df_hourly))]
        if include_ev:
            ev_charge_values = [ev_charge[i].solution_value() for i in range(len(df_hourly))]
            ev_soc_values = [ev_soc[i].solution_value() for i in range(len(df_hourly))]
        else:
            ev_charge_values = None
            ev_soc_values = None

        df_results = pd.DataFrame({'ts':df_hourly['date_hour'],
                                'consumption_wh':df_hourly['consumption_wh'],
                                'production_wh':df_hourly['production_wh'],
                                'battery_charge':battery_charge_values,
                                'battery_discharge':battery_discharge_values,
                                'battery_soc':battery_soc_values,
                                'grid_import':grid_import_values,
                                'grid_export':grid_export_values,
                                'ev_charge':ev_charge_values,
                                'ev_soc':ev_soc_values,
                                'ev_usage':ev_daily_usage_kwh*1000,
                                'tariff':tariffs})
        df_results['site'] = site
        return df_results
    else:
        print(f'No optimal solution found for site {site}')
    time.sleep(0.1)
    del solver

In [12]:
import os

In [13]:
if not os.path.exists('results/tmp/'):
    os.makedirs('results/tmp/')
    scenario_results = []
    for site in useful_sites['site']:
        df_results = get_optimal_solution(site, df_sites, date_range, resolution, time_of_use, ev_daily_avg_usage_km, ev_daily_usage_std, ev_efficiency, ev_charging_start, ev_charging_end, flat_tariff, feed_in_tariff, peak_tariff, off_peak_tariff, peak_initial_hour, peak_final_hour, battery_soc_0, charge_efficiency, discharge_efficiency, ev_max_power_kw, ev_max_energy_kwh, ev_charge_efficiency, ev_soc_0, M, eps, scale)
        if df_results is not None:
            scenario_results.append(df_results)
            df_results.to_parquet(f'results/tmp/{site}.parquet', engine='pyarrow')
    df_results = pd.concat(scenario_results)
    df_results.to_csv(f'results/{scenario_name}.csv', index=False)
else:
    sites = [int(f.split('.parquet')[0]) for f in os.listdir('results/tmp/')]
    scenario_results = []
    for site in sites:
        df_results = pd.read_parquet(f'results/tmp/{site}.parquet', engine='pyarrow')
        scenario_results.append(df_results)
    for site in useful_sites['site']:
        if site not in sites:
            df_results = get_optimal_solution(site, df_sites, date_range, resolution, time_of_use, ev_daily_avg_usage_km, ev_daily_usage_std, ev_efficiency, ev_charging_start, ev_charging_end, flat_tariff, feed_in_tariff, peak_tariff, off_peak_tariff, peak_initial_hour, peak_final_hour, battery_soc_0, charge_efficiency, discharge_efficiency, ev_max_power_kw, ev_max_energy_kwh, ev_charge_efficiency, ev_soc_0, M, eps, scale)
            if df_results is not None:
                scenario_results.append(df_results)
                df_results.to_parquet(f'results/tmp/{site}.parquet', engine='pyarrow')
    df_results = pd.concat(scenario_results)
    df_results.to_csv(f'results/{scenario_name}.csv', index=False)


Optimal solution found for site 1
Optimal solution found for site 2
Optimal solution found for site 3
Optimal solution found for site 4
Optimal solution found for site 5
Optimal solution found for site 6
Optimal solution found for site 7
No optimal solution found for site 8
Optimal solution found for site 9
Optimal solution found for site 10
Optimal solution found for site 11
Optimal solution found for site 12
Optimal solution found for site 13
Optimal solution found for site 14
Optimal solution found for site 15
Optimal solution found for site 16
No optimal solution found for site 17
Optimal solution found for site 18
Optimal solution found for site 19
Optimal solution found for site 20
Optimal solution found for site 21
Optimal solution found for site 22
No optimal solution found for site 23
Optimal solution found for site 24
Optimal solution found for site 25
Optimal solution found for site 26
Optimal solution found for site 27
No optimal solution found for site 28
Optimal solution 

In [14]:
os.system('rm -r results/tmp/') # this


0